### 피아노 Midi 사이트
- https://www.freepianotutorials.net/2023/12/ludwig-goransson-can-you-hear-music.html#more
- http://www.piano-midi.de/bach.htm

### Ⅰ. 라이브러리

In [2]:
import os
import mido
import numpy as np
import pandas as pd
import editdistance

from mido import MidiFile, MidiTrack, MetaMessage, Message

In [42]:
data_folder = "midi_data"

tick_base = False
sequence_base = True


In [4]:
def load_midi_data(input_name, target_name):
    input_path  = os.path.join(data_folder, input_name)
    target_path = os.path.join(data_folder, target_name)
    
    input_mid  = mido.MidiFile(input_path)
    target_mid = mido.MidiFile(target_path)
    
    #input_mid.tracks  = input_mid.tracks[0:2]
    #target_mid.tracks = target_mid.tracks[0:2]
    
    #######################################################
    ######## merge : tick 때문에 merge 하지 않음
    #######################################################
    
    # input_mid = mido.merge_tracks(input_mid.tracks)
    # target_mid = mido.merge_tracks(target_mid.tracks)
    
    return input_mid, target_mid

In [6]:
input_name = "river_flows_in_you_test_v2.mid"
target_name = "river_flows_in_you_origin.mid"

input_mid, target_mid = load_midi_data(input_name, target_name)

### Ⅱ. 데이터 추출하기
- 데이터 전제
    + track 길이 (2가지) : meta data, msg

In [7]:
def load_mid_Info_from_csv(input_name, target_name):
    input_csv  = input_name.split(".")[0]+".csv"
    target_csv = target_name.split(".")[0]+".csv"
    
    input_info = pd.read_csv(input_csv, index_col = 0)
    target_info = pd.read_csv(target_csv, index_col = 0)
    
    return input_info, target_info

aa, bb = load_mid_Info_from_csv("hello.mid", "hello.mid")

In [30]:
def create_df_info():
    df_col = ['sec', 'bpm', 'tick', 'msg_type', 'channel', 'note', 'velocity', 'count', 'main_vol','depth', 'pedal', 'pan', 'tempo']
    df = pd.DataFrame(columns = df_col)
    
    df_tempo_col = ['tempo', 'tick']
    df_tempo = pd.DataFrame(columns = df_tempo_col)

    df_info = { MetaMessage:{}, Message:df }
    
    return df_info, df_tempo

def info_to_list(cur_info):
    
    cur_temp = [cur_info['sec'], cur_info['bpm'], cur_info['tick'], 
                cur_info['msg_type'], cur_info['channel'], cur_info['note'], cur_info['velocity'], cur_info['count'],
                cur_info['main_vol'], cur_info['depth'], cur_info['pedal'], cur_info['pan'], cur_info['tempo']]
    
    return cur_temp
                

def initialize_cur_info():
    cur_info = {'sec':0, 'bpm':0, 'tick':0, 
                'msg_type': "", 'channel':0, 'note':0,  'velocity':0, 'count':0, 
                'main_vol':-1, 'depth':-1, 'pedal':-1, 'pan':-1, 'tempo':0 }
    return cur_info

def check_not_list(msg_list):
    if type(msg_list) == list:
        return msg_list
    else:
        msg_list = [msg_list]
        return msg_list

In [9]:
def tempo_info_list(tempo_list, tempo_info):
    tempo_tick = 0
    
    if type(tempo_list) != list:
        tempo_info.loc[0, 'tempo'] = tempo_list.tempo
        tempo_info.loc[0, 'tick'] = -1 # -1 : 마지막 숫자
        
    else:
        for idx, msg in enumerate(tempo_list):
            tempo_tick += msg.time
            
            if i==0:
                tempo_info.loc[i,'tempo'] = msg.tempo
            else:
                tempo_info.loc[i,'tempo'] = msg.tempo
                tempo_info.loc[i-1,'tick'] = tempo_tick
            
        tempo_info.loc[tempo_info.shape[0] - 1, 'tick'] = -1
        
    return tempo_info

def find_tempo(tick, tempo_info):
    tempo_idx = 0
    last_tempo_idx = tempo_info.shape[0]
    
    cur_tempo = -1
    
    for tempo_idx in range(last_tempo_idx):
        if tempo_info.loc[tempo_idx, 'tick'] > tick:
            cur_tempo = tempo_info.loc[tempo_idx, 'tempo']
            break
            
    if cur_tempo == - 1:
        cur_tempo = tempo_info.loc[last_tempo_idx - 1, 'tempo']
        
    return cur_tempo

In [10]:
def control_type(msg):
    
    if msg.control == 1:
        ctl_type = 'modulation'
        
    elif msg.control == 7:
        ctl_type = 'main_vol'
        
    elif msg.control == 10:
        ctl_type = 'pan'
        
    elif msg.control == 64:
        ctl_type = 'pedal'
        
    elif msg.control >= 91 and msg.control <= 93:
        ctl_type = 'depth'
        
    return ctl_type

In [11]:
def process_msg(msg, info):
    
    if msg.type == 'note_on' or msg.type == 'note_off':
        
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_type = 'note_off'
        else:
            msg_type = msg.type
            
        if info['count'] == 1:
            info['msg_type'] = [info['msg_type']]
            info['channel']  = [info['channel']]
            info['note']     = [info['note']]
            info['velocity'] = [info['velocity']]
                    
            info['msg_type'].append(msg_type)
            info['channel'].append(msg.channel)
            info['note'].append(msg.note)
            info['velocity'].append(msg.velocity)
                    
        else:
            info['msg_type'] = msg_type
            info['channel'] = msg.channel
            info['note'] = msg.note
            info['velocity'] = msg.velocity
                    
        info['count']+=1
        
    elif msg.type == 'program_change':
        print("program_change")
        
    elif msg.type == 'control_change':
        
        ctl_type = control_type(msg)
        
        if info[ctl_type] == -1:
            info[ctl_type] = msg.value
        else:
            info[ctl_type] = check_not_list(info[ctl_type])
            info[ctl_type].append(msg.value)
    
    return info

In [41]:
def extract_mid_data(mid, file_name):
    
    mid_info, tempo_info = create_df_info()
    
    for idx, track in enumerate(mid.tracks):
        
        cur_tick = 0
        cur_sec  = 0
        cur_info = initialize_cur_info()
        
        for msg in track:
            
            ###############################################
            ########   MetaMessage
            ###############################################
            if isinstance(msg, MetaMessage):
                
                if msg.type not in mid_info[MetaMessage]:
                    mid_info[MetaMessage][msg.type] = msg
                    
                else:
                    mid_info[MetaMessage][msg.type] = check_not_list(mid_info[MetaMessage][msg.type])
                    mid_info[MetaMessage][msg.type].append(msg)
                    
            ###############################################
            ########   Message
            ###############################################
            elif isinstance(msg, Message):
                if msg.time > 0:
                    cur_info['tempo'] = find_tempo(cur_tick, tempo_info)
                    cur_info['tick'] = cur_tick
                    cur_info['sec'] = mido.tick2second( cur_info['tick'], mid.ticks_per_beat, cur_info['tempo'] )
                    cur_info['bpm'] = mido.tempo2bpm(cur_info['tempo'])

                    cur_temp = info_to_list(cur_info)
                    
                    if tick_base:
                        #########################################################
                        #####  tick 기반
                        #########################################################
                                                
                        # insert row info
                        cur_tick = cur_info['tick'] + msg.time
                        last_tick = mid_info[Message].shape[0]
                        for idx in range(last_tick, cur_tick):
                            cur_temp[0] = mido.tick2second(idx, mid.ticks_per_beat, cur_info['tempo'] ) # second
                            cur_temp[1] = mido.tempo2bpm(cur_info['tempo'])
                            cur_temp[2] = idx # tick
                        
                            mid_info[Message].loc[idx] = cur_temp
                    
                    
                    else:
                        #########################################################
                        #####  순서
                        #########################################################
                        print("AAAAAAAAAAAAAAAA")
                    
                        mid_info[Message] = mid_info[Message].append(cur_temp);
                        
                    cur_info = initialize_cur_info()
                
                cur_info = process_msg(msg, cur_info)
                print(msg)
                
                
                    
        # tempo setting
        if idx == 0:
            # 예외 사항
            # set_tempo 가 없을 경우, default : 500000 로 설정
            if 'set_tempo' not in mid_info[MetaMessage]:
                mid_info[MetaMessage][msg.type] = mido.MetaMessage('set_tempo', tempo = 500000)
            
            tempo_info_list(mid_info[MetaMessage]['set_tempo'], tempo_info)
            
        if idx == 100:
            if cur_info['count'] > 0:
                print("Iam here")
                cur_info['tempo'] = find_tempo(cur_tick, tempo_info)
                cur_info['tick'] = cur_tick
                cur_info['sec'] = mido.tick2second( cur_info['tick'], mid.ticks_per_beat, cur_info['tempo'] )
            break
        
        
            
    mid_info[Message].replace(-1, 0, inplace = True)
    
    csv_name = file_name.split(".")[0]+".csv"
    mid_info[Message].to_csv(csv_name)
    
    display(mid_info[Message])
    
    return mid_info


In [34]:
input_info  = extract_mid_data(input_mid, input_name)
# target_info = extract_mid_data(target_mid, target_name)

note_on channel=0 note=80 velocity=36 time=253
note_on channel=0 note=80 velocity=36 time=1
note_on channel=0 note=61 velocity=17 time=4
note_on channel=0 note=61 velocity=17 time=1
note_on channel=0 note=61 velocity=0 time=146
note_on channel=0 note=61 velocity=0 time=0
note_on channel=0 note=81 velocity=53 time=63
note_on channel=0 note=81 velocity=53 time=1
note_on channel=0 note=66 velocity=36 time=8
note_on channel=0 note=66 velocity=36 time=1
note_on channel=0 note=80 velocity=0 time=8
note_on channel=0 note=80 velocity=0 time=0
note_on channel=0 note=66 velocity=0 time=35
note_on channel=0 note=66 velocity=0 time=0
note_on channel=0 note=81 velocity=0 time=147
note_on channel=0 note=81 velocity=0 time=0
note_on channel=0 note=80 velocity=58 time=33
note_on channel=0 note=80 velocity=58 time=1
control_change channel=0 control=64 value=120 time=214
control_change channel=0 control=64 value=120 time=0
control_change channel=0 control=64 value=114 time=3
control_change channel=0 con

note_on channel=0 note=76 velocity=0 time=14
note_on channel=0 note=76 velocity=0 time=0
note_on channel=0 note=81 velocity=0 time=196
note_on channel=0 note=81 velocity=0 time=0
note_on channel=0 note=74 velocity=53 time=21
note_on channel=0 note=74 velocity=53 time=1
note_on channel=0 note=74 velocity=0 time=1434
note_on channel=0 note=74 velocity=0 time=0
note_on channel=0 note=69 velocity=30 time=58
note_on channel=0 note=69 velocity=30 time=1
note_on channel=0 note=62 velocity=0 time=21
note_on channel=0 note=62 velocity=0 time=0
note_on channel=0 note=73 velocity=46 time=80
note_on channel=0 note=73 velocity=46 time=0
note_on channel=0 note=69 velocity=0 time=24
note_on channel=0 note=69 velocity=0 time=0
control_change channel=0 control=64 value=120 time=31
control_change channel=0 control=64 value=120 time=0
control_change channel=0 control=64 value=104 time=5
control_change channel=0 control=64 value=104 time=0
control_change channel=0 control=64 value=86 time=3
control_change

note_on channel=0 note=57 velocity=40 time=29
note_on channel=0 note=57 velocity=40 time=1
note_on channel=0 note=74 velocity=0 time=8
note_on channel=0 note=74 velocity=0 time=0
note_on channel=0 note=76 velocity=51 time=6
note_on channel=0 note=76 velocity=51 time=0
note_on channel=0 note=57 velocity=0 time=66
note_on channel=0 note=57 velocity=0 time=0
note_on channel=0 note=76 velocity=0 time=158
note_on channel=0 note=76 velocity=0 time=0
note_on channel=0 note=73 velocity=65 time=11
note_on channel=0 note=73 velocity=65 time=1
control_change channel=0 control=64 value=124 time=190
control_change channel=0 control=64 value=124 time=1
control_change channel=0 control=64 value=102 time=5
control_change channel=0 control=64 value=102 time=0
control_change channel=0 control=64 value=78 time=4
control_change channel=0 control=64 value=78 time=0
control_change channel=0 control=64 value=56 time=8
control_change channel=0 control=64 value=56 time=0
control_change channel=0 control=64 val

control_change channel=0 control=64 value=90 time=4
control_change channel=0 control=64 value=90 time=0
control_change channel=0 control=64 value=114 time=4
control_change channel=0 control=64 value=114 time=0
control_change channel=0 control=64 value=127 time=5
control_change channel=0 control=64 value=127 time=1
note_on channel=0 note=45 velocity=0 time=163
note_on channel=0 note=45 velocity=0 time=0
note_on channel=0 note=52 velocity=38 time=9
note_on channel=0 note=52 velocity=38 time=0
note_on channel=0 note=57 velocity=46 time=228
note_on channel=0 note=57 velocity=46 time=0
note_on channel=0 note=52 velocity=0 time=6
note_on channel=0 note=52 velocity=0 time=0
note_on channel=0 note=73 velocity=68 time=5
note_on channel=0 note=73 velocity=68 time=1
note_on channel=0 note=76 velocity=0 time=23
note_on channel=0 note=76 velocity=0 time=0
note_on channel=0 note=57 velocity=0 time=59
note_on channel=0 note=57 velocity=0 time=0
note_on channel=0 note=74 velocity=51 time=143
note_on c

note_on channel=0 note=50 velocity=0 time=6
note_on channel=0 note=50 velocity=0 time=0
note_on channel=0 note=69 velocity=0 time=3
note_on channel=0 note=69 velocity=0 time=0
note_on channel=0 note=81 velocity=64 time=109
note_on channel=0 note=81 velocity=64 time=0
note_on channel=0 note=76 velocity=0 time=23
note_on channel=0 note=76 velocity=0 time=1
note_on channel=0 note=62 velocity=51 time=94
note_on channel=0 note=62 velocity=51 time=0
note_on channel=0 note=57 velocity=0 time=43
note_on channel=0 note=57 velocity=0 time=0
note_on channel=0 note=81 velocity=0 time=67
note_on channel=0 note=81 velocity=0 time=0
note_on channel=0 note=69 velocity=55 time=8
note_on channel=0 note=69 velocity=55 time=0
note_on channel=0 note=62 velocity=0 time=6
note_on channel=0 note=62 velocity=0 time=0
note_on channel=0 note=69 velocity=0 time=125
note_on channel=0 note=69 velocity=0 time=0
note_on channel=0 note=74 velocity=59 time=6
note_on channel=0 note=74 velocity=59 time=0
note_on channel=

...

control_change channel=0 control=64 value=34 time=5
control_change channel=0 control=64 value=34 time=0
control_change channel=0 control=64 value=38 time=3
control_change channel=0 control=64 value=38 time=0
control_change channel=0 control=64 value=44 time=5
control_change channel=0 control=64 value=44 time=1
control_change channel=0 control=64 value=56 time=5
control_change channel=0 control=64 value=56 time=0
control_change channel=0 control=64 value=70 time=5
control_change channel=0 control=64 value=70 time=0
note_on channel=0 note=54 velocity=27 time=3
note_on channel=0 note=54 velocity=27 time=1
control_change channel=0 control=64 value=90 time=1
control_change channel=0 control=64 value=90 time=1
note_on channel=0 note=81 velocity=34 time=2
note_on channel=0 note=81 velocity=34 time=0
control_change channel=0 control=64 value=112 time=1
control_change channel=0 control=64 value=112 time=0
control_change channel=0 control=64 value=127 time=5
control_change channel=0 control=64 v

control_change channel=0 control=64 value=127 time=7
control_change channel=0 control=64 value=127 time=0
note_on channel=0 note=81 velocity=0 time=25
note_on channel=0 note=81 velocity=0 time=0
note_on channel=0 note=80 velocity=0 time=223
note_on channel=0 note=80 velocity=0 time=1
note_on channel=0 note=59 velocity=28 time=11
note_on channel=0 note=59 velocity=28 time=1
note_on channel=0 note=52 velocity=0 time=16
note_on channel=0 note=52 velocity=0 time=0
note_on channel=0 note=80 velocity=5 time=39
note_on channel=0 note=80 velocity=5 time=0
note_on channel=0 note=64 velocity=36 time=191
note_on channel=0 note=64 velocity=36 time=1
note_on channel=0 note=59 velocity=0 time=34
note_on channel=0 note=59 velocity=0 time=0
note_on channel=0 note=80 velocity=0 time=28
note_on channel=0 note=80 velocity=0 time=0
note_on channel=0 note=59 velocity=25 time=166
note_on channel=0 note=59 velocity=25 time=0
note_on channel=0 note=64 velocity=0 time=41
note_on channel=0 note=64 velocity=0 ti

control_change channel=0 control=64 value=74 time=6
control_change channel=0 control=64 value=74 time=0
control_change channel=0 control=64 value=72 time=3
control_change channel=0 control=64 value=72 time=0
control_change channel=0 control=64 value=76 time=15
control_change channel=0 control=64 value=76 time=1
control_change channel=0 control=64 value=84 time=5
control_change channel=0 control=64 value=84 time=0
note_on channel=0 note=76 velocity=64 time=3
note_on channel=0 note=76 velocity=64 time=1
control_change channel=0 control=64 value=92 time=0
control_change channel=0 control=64 value=92 time=0
control_change channel=0 control=64 value=106 time=6
control_change channel=0 control=64 value=106 time=0
note_on channel=0 note=54 velocity=36 time=0
note_on channel=0 note=54 velocity=36 time=1
control_change channel=0 control=64 value=122 time=2
control_change channel=0 control=64 value=122 time=0
control_change channel=0 control=64 value=127 time=5
control_change channel=0 control=6

control_change channel=0 control=64 value=127 time=3
control_change channel=0 control=64 value=127 time=0
note_on channel=0 note=54 velocity=32 time=11
note_on channel=0 note=54 velocity=32 time=1
note_on channel=0 note=81 velocity=64 time=3
note_on channel=0 note=81 velocity=64 time=1
note_on channel=0 note=73 velocity=0 time=3
note_on channel=0 note=73 velocity=0 time=0
note_on channel=0 note=54 velocity=0 time=133
note_on channel=0 note=54 velocity=0 time=0
note_on channel=0 note=81 velocity=0 time=87
note_on channel=0 note=81 velocity=0 time=0
note_on channel=0 note=61 velocity=31 time=8
note_on channel=0 note=61 velocity=31 time=0
note_on channel=0 note=80 velocity=56 time=6
note_on channel=0 note=80 velocity=56 time=1
note_on channel=0 note=81 velocity=59 time=129
note_on channel=0 note=81 velocity=59 time=0
note_on channel=0 note=61 velocity=0 time=11
note_on channel=0 note=61 velocity=0 time=0
note_on channel=0 note=80 velocity=0 time=21
note_on channel=0 note=80 velocity=0 tim

,sec,bpm,tick,msg_type,channel,note,velocity,count,main_vol,depth,pedal,pan,tempo
0,0.000000,60.0,0,,0,0,0,0,0,0,0,0,1000000
1,0.002083,60.0,1,,0,0,0,0,0,0,0,0,1000000
2,0.004167,60.0,2,,0,0,0,0,0,0,0,0,1000000
3,0.006250,60.0,3,,0,0,0,0,0,0,0,0,1000000
4,0.008333,60.0,4,,0,0,0,0,0,0,0,0,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83889,174.768750,60.0,83889,,0,0,0,0,0,0,"[1, 1]",0,1000000
83890,174.770833,60.0,83890,,0,0,0,0,0,0,"[1, 1]",0,1000000
83891,174.772917,60.0,83891,,0,0,0,0,0,0,"[1, 1]",0,1000000
83892,174.775000,60.0,83892,,0,0,0,0,0,0,"[1, 1]",0,1000000


In [43]:
input_info  = extract_mid_data(input_mid, input_name)

AAAAAAAAAAAAAAAA
note_on channel=0 note=80 velocity=36 time=253
AAAAAAAAAAAAAAAA
note_on channel=0 note=80 velocity=36 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=61 velocity=17 time=4
AAAAAAAAAAAAAAAA
note_on channel=0 note=61 velocity=17 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=61 velocity=0 time=146
note_on channel=0 note=61 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=81 velocity=53 time=63
AAAAAAAAAAAAAAAA
note_on channel=0 note=81 velocity=53 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=66 velocity=36 time=8
AAAAAAAAAAAAAAAA
note_on channel=0 note=66 velocity=36 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=80 velocity=0 time=8
note_on channel=0 note=80 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=66 velocity=0 time=35
note_on channel=0 note=66 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=81 velocity=0 time=147
note_on channel=0 note=81 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=80 velocity=58 time=33
AAAAAAAAAAAAA

control_change channel=0 control=64 value=118 time=5
control_change channel=0 control=64 value=118 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=124 time=2
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=124 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=127 time=5
control_change channel=0 control=64 value=127 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=81 velocity=0 time=183
note_on channel=0 note=81 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=76 velocity=48 time=25
note_on channel=0 note=76 velocity=48 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=57 velocity=35 time=4
AAAAAAAAAAAAAAAA
note_on channel=0 note=57 velocity=35 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=50 velocity=0 time=3
note_on channel=0 note=50 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=57 velocity=0 time=218
note_on channel=0 note=57 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=62 velocity=30 time=9
note_on channel=0 

note_on channel=0 note=50 velocity=0 time=112
note_on channel=0 note=50 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=57 velocity=44 time=1
note_on channel=0 note=57 velocity=44 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=76 velocity=59 time=5
AAAAAAAAAAAAAAAA
note_on channel=0 note=76 velocity=59 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=0 time=15
note_on channel=0 note=69 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=81 velocity=64 time=108
AAAAAAAAAAAAAAAA
note_on channel=0 note=81 velocity=64 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=76 velocity=0 time=28
note_on channel=0 note=76 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=62 velocity=42 time=66
AAAAAAAAAAAAAAAA
note_on channel=0 note=62 velocity=42 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=57 velocity=0 time=19
note_on channel=0 note=57 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=62 velocity=0 time=80
note_on channel=0 note=62 velocity=0 time=0
AAAA

note_on channel=0 note=69 velocity=0 time=8
note_on channel=0 note=69 velocity=0 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=122 time=86
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=122 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=94 time=2
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=94 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=64 time=5
control_change channel=0 control=64 value=64 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=32 time=7
control_change channel=0 control=64 value=32 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=10 time=3
control_change channel=0 control=64 value=10 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=0 time=5
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=0 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=50 velocity=44 time=5
note_on channel=0 note=50 velocity=44 time=0
AAAAAAAAAAAAAAAA
note_on ch

note_on channel=0 note=59 velocity=0 time=44
note_on channel=0 note=59 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=48 time=179
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=48 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=64 velocity=0 time=58
note_on channel=0 note=64 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=73 velocity=56 time=69
note_on channel=0 note=73 velocity=56 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=0 time=30
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=0 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=124 time=73
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=124 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=104 time=5
control_change channel=0 control=64 value=104 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=82 time=5
control_change channel=0 control=64 value=82 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=62 t

...

control_change channel=0 control=64 value=14 time=9
control_change channel=0 control=64 value=14 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=1 time=5
control_change channel=0 control=64 value=1 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=0 time=6
control_change channel=0 control=64 value=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=74 velocity=66 time=3
AAAAAAAAAAAAAAAA
note_on channel=0 note=74 velocity=66 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=1 time=5
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=1 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=6 time=3
control_change channel=0 control=64 value=6 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=16 time=5
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=16 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=28 time=4
control_change channel=0 control=64 value=28 time=0
AAAAAAAAAAAAAAAA
con

control_change channel=0 control=64 value=90 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=80 time=5
control_change channel=0 control=64 value=80 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=74 time=6
control_change channel=0 control=64 value=74 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=72 time=3
control_change channel=0 control=64 value=72 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=76 time=15
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=76 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=84 time=5
control_change channel=0 control=64 value=84 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=76 velocity=64 time=3
AAAAAAAAAAAAAAAA
note_on channel=0 note=76 velocity=64 time=1
control_change channel=0 control=64 value=92 time=0
control_change channel=0 control=64 value=92 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=106 time=6
control_change channel=0 co

note_on channel=0 note=73 velocity=0 time=3
note_on channel=0 note=73 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=71 velocity=0 time=207
AAAAAAAAAAAAAAAA
note_on channel=0 note=71 velocity=0 time=1
note_on channel=0 note=64 velocity=36 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=64 velocity=36 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=57 velocity=0 time=2
note_on channel=0 note=57 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=44 time=247
note_on channel=0 note=69 velocity=44 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=64 velocity=0 time=23
note_on channel=0 note=64 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=0 time=50
note_on channel=0 note=69 velocity=0 time=0
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=51 time=166
AAAAAAAAAAAAAAAA
note_on channel=0 note=69 velocity=51 time=1
AAAAAAAAAAAAAAAA
note_on channel=0 note=73 velocity=48 time=125
AAAAAAAAAAAAAAAA
note_on channel=0 note=73 velocity=48 time=1
AAA

control_change channel=0 control=64 value=108 time=5
control_change channel=0 control=64 value=108 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=96 time=5
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=96 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=86 time=5
control_change channel=0 control=64 value=86 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=74 time=3
control_change channel=0 control=64 value=74 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=64 time=5
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=64 time=1
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=54 time=6
control_change channel=0 control=64 value=54 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=42 time=3
control_change channel=0 control=64 value=42 time=0
AAAAAAAAAAAAAAAA
control_change channel=0 control=64 value=34 time=6
control_change channel=0 control=64 value=34 time=0
AAAAAAAAAAAAA

,0,bpm,channel,count,depth,main_vol,msg_type,note,pan,pedal,sec,tempo,tick,velocity
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,"[1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### [0]. precheck
- Mid Data 전제 조건
    + track 2개 : [ Metadata + Midi ], [ Piano ]
    + track_name 통일

In [14]:
def pre_check(input_mid, target_mid):
    input_track_len  = len(input_mid.tracks)
    target_track_len = len(target_mid.tracks)
    
    # track 길이 비교
    if input_track_len != target_track_len:
        print(f"input track len({input_track_len}) != target track len({target_track_len})")
        return False, None, None
    
    # track_name 비교
    input_track_name  = [ msg.name for msg in input_mid if msg.type == 'track_name']
    target_track_name = [ msg.name for msg in target_mid if msg.type == 'track_name']
    
    ## if input_track_name != target_track_name:
    if input_track_name == target_track_name:
        print(f"input track name({input_track_name}) != target track name({target_track_name})")
        return False, None, None
    
    input_info  = extract_mid_data(input_mid, input_name)
    target_info = extract_mid_data(target_mid, target_name)
    
    # 시간 비교
    #############################################################
    ############ 순서, 시간 차이를 어떻게 할 것인지 고민
    #############################################################
    #if input_info[Message].shape[0] != target_info[Message].shape[0]:
    #    return False
    
    return True, input_info, target_info

bRet, input_info, target_info = pre_check(input_mid, target_mid)

NameError: name 'cur_temp' is not defined

#### [1]. MetaMessage
- (1). midi_port ( 보류 )
- (2). set_tempo
- (3). smpte_offset
- (4). time_signature
- (5). key_signature

In [357]:
def compare_MetaMessage(input_meta, target_meta):
    
    score = 0
    total_score = 0 # 비교 해야 할 대상의 갯수
    
    for key in input_meta:
        
        if key == 'midi_port':
            count = 0
            
            if type(input_meta[key]) == list:
                count += len(input_meta[key])
            else:
                count +=1
                
            total_score +=count
                
            if key not in target_meta: continue
                
            if count > 1: # list
                for idx in range(count):
                    if input_meta[key][idx].port == target_meta[key][idx].port : score+=1
            else:
                if input_meta[key].port == target_meta[key].port : score+=1
            
        elif key == 'set_tempo':
            # Message 에서 비교
            continue
            
        elif key == 'smpte_offset':
            # frame_rate, hours, minutes, seconds, frames, sub_frames
            total_score += 6
            
            if key not in target_meta: continue
            
            if input_meta[key].frame_rate == target_meta[key].frame_rate: score+=1
            if input_meta[key].hours == target_meta[key].hours: score+=1
            if input_meta[key].minutes == target_meta[key].minutes: score+=1   
            if input_meta[key].seconds == target_meta[key].seconds: score+=1   
            if input_meta[key].frames == target_meta[key].frames: score+=1     
            if input_meta[key].sub_frames == target_meta[key].sub_frames: score+=1
            
        elif key == 'time_signature':
            # numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8
            total_score += 4
            
            if key not in target_meta: continue
            
            if input_meta[key].numerator == target_meta[key].numerator: score+=1
            if input_meta[key].denominator == target_meta[key].denominator: score+=1
            if input_meta[key].clocks_per_click == target_meta[key].clocks_per_click: score+=1 
            if input_meta[key].notated_32nd_notes_per_beat == target_meta[key].notated_32nd_notes_per_beat: score+=1
            
        elif key == 'key_signature':
            # numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8
            total_score += 1
            
            if key not in target_meta: continue
                
            if input_meta[key].key == target_meta[key].key: score+=1  
        
        elif key == 'text':
            count = 0
            
            if type(input_meta[key]) == list:
                count += len(input_meta[key])
            else:
                count +=1
                
            total_score +=count
                
            if key not in target_meta: continue
                
            if count > 1: # list
                for idx in range(count):
                    if input_meta[key][idx].text == target_meta[key][idx].text : score+=1
            else:
                if input_meta[key].text == target_meta[key].text : score+=1
            
            
        else:
            continue
            
    print(f"MetaMessage : score({score}), total_score({total_score})")
    print(f"MetaMessage : percentage({score/total_score*100})")
            
    return score, total_score

In [358]:
compare_MetaMessage(input_info[MetaMessage], target_info[MetaMessage])

MetaMessage : score(2), total_score(7)
MetaMessage : percentage(28.57142857142857)


(2, 7)

#### [2]. Message

In [ ]:
def compare_Message(input_msg, target_msg):
    end_tick = input_msg.shape[0]
    
    for start_tick in range(end_tick):
        

In [ ]:
compare_Message(input_info[Message], target_info[Message]):

In [10]:
import mido 
mid1 = mido.MidiFile("midi_data/river_flows_in_you_origin.mid")
mid2 = mido.MidiFile("midi_data/river_flows_in_you_test_v2.mid")

In [15]:
from tqdm.notebook import tqdm